In [ ]:
import torch
import numpy as np
from rfm import LaplaceRFM
torch.set_default_dtype(torch.float32)
torch.manual_seed(0)
# define target function
def fstar(X):
    return torch.cat([
        (X[:, 0]  > 0)[:,None],
        (X[:, 1]  < 0.1)[:,None]],
        axis=1).type(X.type())

# create low rank data
n = 4000
d = 100
torch.manual_seed(0)
X_train = torch.randn(n,d)
X_test = torch.randn(n,d)

y_train = fstar(X_train)
y_test = fstar(X_test)

model = LaplaceRFM(bandwidth=1., diag=False, centering=False)
model.fit(
    (X_train, y_train), 
    (X_test, y_test), 
     method='nmf', epochs=15,
    iters=5,
    classification=True,
    verbose=True
) 


Time taken to prefit nmf with 70000 points: 0.3796577453613281 seconds


TypeError: KernelModel.fit() got an unexpected keyword argument 'classif'